# CP467 Assignment 1

The input images are put in the Images folder. Output images are saved in the Results folder and seperated by the questions (i.e. the output images of questions 1 are saved in Results/Question 1 and output images for question 2 are saved in Results/Question 2, etc.)

## Image Interpolation

Resize the lena image so that it can be resized and interpolated

In [17]:
import cv2

# Load the image using OpenCV
image = cv2.imread('Images/lena.tif')

# Get original dimensions
height, width = image.shape[:2]

# Calculate new dimensions (1/2 of the original in each dimension)
down_width = width // 2
down_height = height // 2

# Resize the image
downscaled_image = cv2.resize(image, (down_width, down_height))

Nearest neighbor interpolation implementation from scratch

In [18]:
import numpy as np

# Create an empty matrix for the rescaled image (original dimensions)
rescaled_image_np = np.zeros((height, width, 3), dtype=downscaled_image.dtype)

# Nearest-neighbor interpolation
for i in range(height):
    for j in range(width):
        # Find the nearest pixel in the downscaled image
        nearest_i = int(i / 2)
        nearest_j = int(j / 2)

        # Assign the nearest pixel value to the rescaled image
        rescaled_image_np[i, j] = downscaled_image[nearest_i, nearest_j]

# Save the rescaled image using OpenCV
cv2.imwrite('Results/Question 1/lena_nearest_scratch.tif', rescaled_image_np)

True

Nearest neighbor interpolation using OpenCV

In [19]:
upscaled_image_cv = cv2.resize(downscaled_image, (width, height), interpolation=cv2.INTER_NEAREST)

# Save the upscaled image using OpenCV
cv2.imwrite('Results/Question 1/lena_nearest_cv.tif', upscaled_image_cv)

True

Bilinear Interpolation from Scratch

In [28]:
downscaled_array = np.array(downscaled_image)

# Initialize an empty array for the upscaled image
rescaled_image_np = np.zeros((height, width, 3), dtype=downscaled_image.dtype)

# Calculate scaling factors
scale_x = down_width / width
scale_y = down_height / height

# Perform bilinear interpolation
for i in range(height):
    for j in range(width):
        # Corresponding position in the downscaled image
        x = j * scale_x
        y = i * scale_y

        # Coordinates of the top-left pixel
        x0 = int(np.floor(x))
        y0 = int(np.floor(y))

        # Coordinates of the bottom-right pixel
        x1 = min(x0 + 1, down_width - 1)
        y1 = min(y0 + 1, down_height - 1)

        # Calculate the distance between the target position and the neighboring pixels
        dx = x - x0
        dy = y - y0

        # Retrieve pixel values
        Ia = downscaled_array[y0, x0]
        Ib = downscaled_array[y1, x0]
        Ic = downscaled_array[y0, x1]
        Id = downscaled_array[y1, x1]

        # Compute interpolated value

        # Get weighted average contribution for each neighbour
        top_left_pixel = (1 - dx) * (1 - dy) * Ia
        top_right_pixel = dx * (1 - dy) * Ic
        bottom_left_pixel = (1 - dx) * dy * Ib
        bottom_right_pixel = dx * dy * Id

        # Calculate weighted average
        weighted_average = top_left_pixel + top_right_pixel + bottom_left_pixel + bottom_right_pixel
    
        rescaled_image_np[i, j] = np.clip(weighted_average, 0, 255)

cv2.imwrite('Results/Question 1/lena_bilinear_scratch.tif', rescaled_image_np)

True

Bilinear Interpolation using OpenCV

In [30]:
upscaled_image_cv = cv2.resize(downscaled_image, (width, height), interpolation=cv2.INTER_LINEAR)

# Save the upscaled image using OpenCV
cv2.imwrite('Results/Question 1/lena_bilinear_cv.tif', upscaled_image_cv)

True

Bicubic Interpolation from scratch

In [6]:
def cubic_kernel(x, a=-0.5):
    """
    Cubic kernel function for bicubic interpolation.
    
    Parameters:
        x (float): The distance from the central pixel.
        a (float): Parameter for the cubic function. Commonly -0.5 (Catmull-Rom).
        
    Returns:
        float: Weight for the given distance.
    """
    abs_x = np.abs(x)
    if abs_x <= 1:
        return (a + 2) * abs_x**3 - (a + 3) * abs_x**2 + 1
    elif 1 < abs_x < 2:
        return a * abs_x**3 - 5*a * abs_x**2 + 8*a * abs_x - 4*a
    else:
        return 0

In [7]:
def bicubic_interpolate(image, scale):
    """
    Rescales the image using bicubic interpolation.
    
    Parameters:
        image (numpy.ndarray): Input image as a NumPy array.
        scale (float): Scaling factor (e.g., 2 for doubling the size).
        
    Returns:
        numpy.ndarray: Upscaled image.
    """
    if len(image.shape) == 2:  # Grayscale
        channels = 1
    else:
        channels = image.shape[2]
    
    in_height, in_width = image.shape[:2]
    out_height, out_width = int(in_height * scale), int(in_width * scale)
    
    # Prepare the output image
    if channels == 1:
        output = np.zeros((out_height, out_width), dtype=image.dtype)
    else:
        output = np.zeros((out_height, out_width, channels), dtype=image.dtype)
    
    # Precompute the scaled coordinates
    scale_inv = 1 / scale
    for y in range(out_height):
        for x in range(out_width):
            # Map the coordinates back to the input image
            src_x = x * scale_inv
            src_y = y * scale_inv
            
            x_int = int(np.floor(src_x))
            y_int = int(np.floor(src_y))
            
            x_frac = src_x - x_int
            y_frac = src_y - y_int
            
            # Accumulate the results
            for m in range(-1, 3):
                for n in range(-1, 3):
                    # Neighbor coordinates
                    neighbor_x = x_int + n
                    neighbor_y = y_int + m
                    
                    # Handle boundary conditions
                    neighbor_x = min(max(neighbor_x, 0), in_width - 1)
                    neighbor_y = min(max(neighbor_y, 0), in_height - 1)
                    
                    weight = cubic_kernel(m - y_frac) * cubic_kernel(n - x_frac)
                    
                    if channels == 1:
                        output[y, x] += image[neighbor_y, neighbor_x] * weight
                    else:
                        output[y, x, :] += image[neighbor_y, neighbor_x, :] * weight
    
    # Clip the values to valid range
    output = np.clip(output, 0, 255)
    output = output.astype(image.dtype)
    
    return output

In [8]:
downscaled_image = np.array(downscaled_image).astype(np.float32)
    
# Check if the image has an alpha channel and remove it if present
if downscaled_image.ndim == 3 and downscaled_image.shape[2] == 4:
    downscaled_image = downscaled_image[:, :, :3]

# Define the scaling factor (2x in each dimension)
scale_factor = 2.0

# Perform bicubic interpolation
image = bicubic_interpolate(downscaled_image, scale_factor)
upscaled_image = Image.fromarray(image.astype(np.uint8))
upscaled_image.save('Results/Question 1/lena_bicubic_scratch.tif')

Bicubic Interpolation using OpenCV

In [9]:
# Use OpenCV's resize function with INTER_CUBIC for bicubic interpolation
upscaled_image_cv = cv2.resize(downscaled_image_cv, (width, height), interpolation=cv2.INTER_CUBIC)

bicubic_cv_image = Image.fromarray(cv2.cvtColor(upscaled_image_cv, cv2.COLOR_BGR2RGB))
bicubic_cv_image.save('Results/Question 1/lena_bicubic_cv.tif')

Quantitively compare images (MSE)

In [10]:
def compute_mse(image1_path, image2_path):
    """
    Computes the Mean Squared Error between two images.

    Parameters:
        image1_path (str): Path to the first image (original image).
        image2_path (str): Path to the second image (interpolated image).

    Returns:
        float: The MSE between the two images.
    """
    img1 = Image.open(image1_path).convert('RGB')
    img2 = Image.open(image2_path).convert('RGB')

    img1_np = np.array(img1, dtype=np.float64)
    img2_np = np.array(img2, dtype=np.float64)

    # Ensure the images have the same dimensions
    if img1_np.shape != img2_np.shape:
        raise ValueError(f"Image dimensions do not match: {img1_np.shape} vs {img2_np.shape}")

    # Compute the Mean Squared Error
    mse = np.mean((img1_np - img2_np) ** 2)
    return mse

In [11]:
resultsPath = 'Results/Question 1/'
original_image_path = 'Images/lena.tif'

interpolated_images = {
    'Nearest Neighbor (Scratch)': f'{resultsPath}lena_nearest_scratch.tif',
    'Nearest Neighbor (OpenCV)': f'{resultsPath}lena_nearest_cv.tif',
    'Bilinear (Scratch)': f'{resultsPath}lena_bilinear_scratch.tif',
    'Bilinear (OpenCV)': f'{resultsPath}lena_bilinear_cv.tif',
    'Bicubic (Scratch)': f'{resultsPath}lena_bicubic_scratch.tif',
    'Bicubic (OpenCV)': f'{resultsPath}lena_bicubic_cv.tif'
}

# Compute and print the MSE for each interpolated image
print("Mean Squared Error between Original and Interpolated Images:")
for method, image_path in interpolated_images.items():
    try:
        mse = compute_mse(original_image_path, image_path)
        print(f"{method}: MSE = {mse:.2f}")
    except Exception as e:
        print(f"{method}: Error - {e}")

Mean Squared Error between Original and Interpolated Images:
Nearest Neighbor (Scratch): MSE = 118.98
Nearest Neighbor (OpenCV): MSE = 118.98
Bilinear (Scratch): MSE = 143.44
Bilinear (OpenCV): MSE = 93.95
Bicubic (Scratch): MSE = 139.06
Bicubic (OpenCV): MSE = 74.55


## Point Operations

Get Negative Image

In [12]:
image_path = 'Images/cameraman.tif'
img = Image.open(image_path).convert('L')
img_np = np.array(img)

# Negative of the Image
def negative_image(image_np):
    return 255 - image_np

# Save the negative image
resultsPath = 'Results/Question 2/'
cameraman_negative = negative_image(img_np)
Image.fromarray(cameraman_negative).save(f'{resultsPath}cameraman_negative.tif')

Power-Law Transformation

In [13]:
# Power-Law Transformation (Gamma Correction)
def power_law_transformation(image_np, gamma, c=1):
    # Normalize the image to the range [0, 1]
    normalized_img = image_np / 255.0

    # Apply the power-law transformation
    transformed_img = c * np.power(normalized_img, gamma)

    # Scale back to [0, 255] and cast to uint8
    return np.uint8(transformed_img * 255)

gamma_value = 1.5
cameraman_power = power_law_transformation(img_np, gamma_value)
Image.fromarray(cameraman_power).save(f'{resultsPath}cameraman_power.tif')

Bit-Plane Slicing

In [14]:
# Bit-Plane Slicing
def bit_plane_slicing(image_np, bit):
    # Shift the bits and apply bitwise AND with 1 to extract the bit-plane
    return (image_np >> bit) & 1

# Save bit-plane images
for i in range(8):
    bit_plane_image = bit_plane_slicing(img_np, i) * 255  # Scale to [0, 255]
    Image.fromarray(np.uint8(bit_plane_image)).save(f'{resultsPath}cameraman_b{i+1}.tif')

## Histogram Processing

Histogram Equalization

In [15]:
image_path = 'Images/einstein.tif'
img = Image.open(image_path).convert('L')
img_np = np.array(img)

# Compute the histogram
hist, bins = np.histogram(img_np.flatten(), bins=256, range=[0,256])
cdf = hist.cumsum()
cdf_normalized = cdf * 255 / cdf[-1]  # Scale to [0,255]

# Use linear interpolation of the CDF to find new pixel values
img_equalized_np = np.interp(img_np.flatten(), bins[:-1], cdf_normalized)

# Reshape the flattened array back to the original image shape
img_equalized_np = img_equalized_np.reshape(img_np.shape).astype('uint8')

img_equalized = Image.fromarray(img_equalized_np)
img_equalized.save('Results/Question 3/einstein_equalized.tif')

Histogram Matching

In [16]:
def histogram_matching(source, template):
    """
    Adjust the pixel values of a grayscale image such that its histogram
    matches that of a target image.

    Parameters:
    - source: NumPy array of the source image
    - template: NumPy array of the template image (reference)

    Returns:
    - matched: NumPy array of the transformed output image
    """
    # Flatten the images
    source_flat = source.ravel()
    template_flat = template.ravel()

    # Get the set of unique pixel values and their corresponding indices and counts
    _, bin_idx, s_counts = np.unique(source_flat, return_inverse=True, return_counts=True)
    t_values, t_counts = np.unique(template_flat, return_counts=True)

    # Compute the normalized CDFs
    s_quantiles = np.cumsum(s_counts).astype(np.float64) / source_flat.size
    t_quantiles = np.cumsum(t_counts).astype(np.float64) / template_flat.size

    # Create interpolation function (inverse CDF of the template)
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    # Map the pixel values of the source image to the template
    matched = interp_t_values[bin_idx].reshape(source.shape).astype('uint8')

    return matched

# Load the source images
source_image_path = 'Images/chest_x-ray1.jpeg'
template_image_path = 'Images/chest_x-ray2.jpeg'

source_img = Image.open(source_image_path).convert('L')
template_img = Image.open(template_image_path).convert('L')

source_np = np.array(source_img)
template_np = np.array(template_img)

# Apply histogram matching
matched_np = histogram_matching(source_np, template_np)

# Save the output image
matched_img = Image.fromarray(matched_np)
matched_img.save('Results/Question 3/chest_x-ray3.jpeg')